In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os as os


def getResult(x):
    if((x.盘点差>=-0.15) & (x.盘点差<0.05)):
        return '合格'
    elif(((x.盘点差>=-1.5) & (x.盘点差<-0.15)) || ((x.盘点差>=1) && (x.盘点差<0.05)))
        return '有差'
    elif((x.盘点差<-1.5) | (x.盘点差>1))
        return '差大'
    
pd.set_option('max_columns', 20)
pd.set_option('max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # 禁用科学计数法
#pd.set_option('display.max_rows',None)

#打开文件
df_ERP_output_stock1 = pd.read_excel(r'I:\\#Data\\201912\\20191228\\12限额领料单列表.XLS',usecols=[2,3,6,4]).fillna(0)
df_ERP_output_stock2 = pd.read_excel(r'I:\\#Data\\201912\\20191228\\1限额领料单列表.XLS',usecols=[2,3,4,6]).fillna(0)
df_bar_output_stock1 = pd.read_excel(r'I:\\#Data\\201912\\20191228\\1领料出库单列表.XLSx',usecols=[6,10]).fillna(0)
df_erp_inv = pd.read_excel(r'I:\\#Data\\201912\\20191228\\库存展望.XLS',usecols=[6,7,8,9,10,11]).fillna(0)
df_bar_inv = pd.read_excel(r'I:\\#Data\\201912\\20191228\\库存汇总表.xlsx',usecols=[0,6])
df_stagnate = pd.read_excel(r'I:\\#Data\\201912\\20191228\\盘点完成.xlsx',usecols=[0,4,7])
#df_inv_check = pd.read_excel(r'I:\\#Data\\201912\\20191228\\库存盘点单列表.xlsx',usecols=[0,1,2,4,5,6,7,8,9])
#df_erp_inv=df_erp_inv.iloc[0:10]
df11 = pd.pivot_table(df_ERP_output_stock1,values='计划出库数量',index='材料编码',aggfunc='sum')
df12 = pd.pivot_table(df_ERP_output_stock2,values='计划出库数量',index='材料编码',aggfunc='sum')
    
#df1是ERP结存
df1 = pd.merge(df_erp_inv,df11,left_on='存货编码',right_on='材料编码',how='left').fillna(0)
df1 = pd.merge(df1,df12,left_on='存货编码',right_on='材料编码',how='left').fillna(0)
df1['现存量']=df1['现存量'].astype('float')
#df['结存'] = df[['现存量','计划出库数量_x','计划出库数量_y']].apply(lambda x:x['现存量']-x['计划出库数量_X']-x['计划出库数量_y'],axis=1)
df1['结存'] = df1.apply(lambda x:x.现存量-x.计划出库数量_x-x.计划出库数量_y,axis=1)
#df1
#########################################################################
    
#1月份出库合计
df21 =pd.pivot_table(df_bar_output_stock1,values='计划数量',index='资材编号',aggfunc='sum')
    
#12月份条码结存
df2 = pd.merge(df_bar_inv,df21,on='资材编号',how='left').fillna(0)
df2['结存'] = df2.apply(lambda x:x.期末数量+x.计划数量,axis=1)
    
##########################################################################
df0 = pd.merge(df1,df2,left_on='存货编码',right_on='资材编号',how='outer')
df = pd.merge(df0,df_stagnate,on='存货编码',how='left').fillna(0)
df['出库率'] = df.apply(lambda x:'高' if((x.计划出库数量_x+x.计划出库数量_y)>0) else '低',axis=1)
df['盘点差'] = df.apply(lambda x:0 if(x.结存_x==0) else (x.结存_x-x.结存_y)/x.结存_x,axis=1)
df['结论'] = df.apply(lambda x:getResult(x),axis=1)
df = df[['存货编码','存货名称','规格型号','单位','库管员','结存_x','结存_y','出库率','盘点差','现存量_y','类别','结论']]
df

###########################################################################


SyntaxError: invalid syntax (<ipython-input-17-a9468e5f6757>, line 10)

In [15]:
df[df['类别']=='AR-SB'].sort_values(by=['盘点差'],ascending=True)


,存货编码,存货名称,规格型号,单位,库管员,结存_x,结存_y,出库率,盘点差,现存量_y,类别,结论
2191,JJ48G20724,逆变变压器,22-7788,PCS,黄明喜,1.00,12.00,低,-11.00,12.00,AR-SB,JJ48G20724
5497,MM79G00021,塑料袋,ZXP20880-04,PCS,蒋鹏,4.00,20.00,高,-4.00,20.00,AR-SB,MM79G00021
2192,JJ48G20735,DC电抗器,21-7910 AR-SB200-2,PCS,黄明喜,3.00,9.00,低,-2.00,5.00,AR-SB,JJ48G20735
2112,JJ47D70402,线路板基板支架,BSB-315E,PCS,齐华香,38.00,82.00,高,-1.16,86.00,AR-SB,JJ47D70402
2190,JJ48G20702,滤波线圈,10-9312,PCS,黄明喜,11.00,22.00,低,-1.00,22.00,AR-SB,JJ48G20702
...,...,...,...,...,...,...,...,...,...,...,...,...
1189,JJ22D90234,硅清漆管,1.5ﾏﾙX1M HG-2E,M,吕永全,0.07,0.00,低,1.00,0.00,AR-SB,JJ22D90234
632,JC30A25923,圆头螺栓(TY),M5X16-ZM3CSA,PCS,邢宝华,695.00,0.00,高,1.00,742.00,AR-SB,JC30A25923
1424,JJ35A10370,海绵胶条,7X10X30(ﾃｰﾌﾟﾂｷ),PCS,齐华香,-60.00,89.00,高,2.48,90.00,AR-SB,JJ35A10370
2119,JJ47D80076,橡胶脚,C-30RK-3225,PCS,谈文丽,-10.00,52.00,低,6.20,52.00,AR-SB,JJ47D80076
